### Welcome to the program

This notebook has the following cells
- Parameter file selection
- Dataset and parameter file loading
- Ground truth, prediction, and feature selection
- Metric selection
- Results
- Saving parameters to file

It is recommended to run each cell and fill out the selections before running the next cell. There should be no need to change any code, all of the selection is done by [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/how-to/index.html).

In [ ]:
import utils
import os

import warnings
import torch

import ipywidgets as widgets
from ipywidgets import Layout, Box, VBox, Accordion, Label, Valid, Text, IntText, Combobox, Checkbox, RadioButtons, Button
from IPython.display import display

## part of the project
from fairness_metrics.Predicted_outcomes.Error_rate_metrics import Error_rate_metrics
from fairness_metrics.Predicted_outcomes.Predictive_value_metrics import Predictive_value_metrics
from fairness_metrics.Predicted_outcomes.statistical_parity import statistical_parity
from fairness_metrics.predicted_probs.balance_in_pos_neg import balance_in_pos_neg
from fairness_metrics.predicted_probs.well_callibrated import well_calibration
from fairness_metrics.similarity_based.similarity_based import LipschitzFairness

# warnings.filterwarnings('ignore')

In [ ]:
# Input file selection
INPUT_FILE_NAME = Text(
    placeholder='Type file name',
    description='File name:',
    tooltip='do not forget the file extension'
)
PRESET_FILENAME = INPUT_FILE_NAME.value if INPUT_FILE_NAME.value else 'No file'
file_exits = os.path.exists(utils.SAVED_PRESET_PATH + PRESET_FILENAME)
FILE_FOUND = Valid(
    value=file_exits,
    style = {'description_width': 'initial'},
    description=f'Preset file "{PRESET_FILENAME}"',
    tooltip='Indicates if the chosen file has been found'
)
USE_PRESET= RadioButtons(
    options=[('Yes', 1), ('No', 0)],
    value=file_exits,
    disabled=not(file_exits),
    style = {'description_width': 'initial'},
    description='Load params file:',
)

INPUT_DATASET_NAME = Text(
    placeholder='Type file name',
    description='File name:',
    tooltip='do not forget the file extension'
)
PRESET_DATASET_FILENAME = INPUT_DATASET_NAME.value if INPUT_DATASET_NAME.value else 'No file'
file_exits = os.path.exists(utils.SAVED_PRESET_PATH + PRESET_DATASET_FILENAME)
DATASET_FOUND = Valid(
    value=file_exits,
    style = {'description_width': 'initial'},
    description=f'Preset file "{PRESET_DATASET_FILENAME}"',
    tooltip='Indicates if the chosen file has been found'
)


info_box = VBox([
    Label(value=f'Select your params file here, They must be in the folder "{utils.SAVED_PRESET_PATH}"'),
    INPUT_FILE_NAME,
    FILE_FOUND,
    USE_PRESET,
    INPUT_DATASET_NAME,
    DATASET_FOUND,
])

output_file_selection = widgets.Output()
display(info_box, output_file_selection)

def update_file_name_info(_):
    with output_file_selection:
        PRESET_FILENAME = INPUT_FILE_NAME.value if INPUT_FILE_NAME.value else 'No file'
        file_exits = os.path.exists(os.path.join(utils.SAVED_PRESET_PATH + PRESET_FILENAME))
        FILE_FOUND.description = f'Preset file "{PRESET_FILENAME}"'
        FILE_FOUND.value = file_exits
        USE_PRESET.value = file_exits
        USE_PRESET.disabled=not(file_exits)

INPUT_FILE_NAME.on_trait_change(update_file_name_info)

Output()

In [3]:
# Data and file loading
# dataset = utils.Dataset(utils.DATA_PATH_SYNTH)
dataset = utils.Dataset('data/altered_data/alt_data_balenced_pred_biased.csv')
display(Label(value='dataset has been loaded'))

params = {}
if USE_PRESET.value:
    params = torch.load(utils.SAVED_PRESET_PATH + INPUT_FILE_NAME.value, weights_only=True)

    display(Label(value=f'"{INPUT_FILE_NAME.value}" has been loaded'))

torch.save(params, utils.SAVED_PRESET_PATH + utils.RESEVERD_PRESET)


Label(value='dataset has been loaded')

Label(value='"save.pt" has been loaded')

In [4]:
# ground truth, prediction, and protected atrributes selection.
params = torch.load(utils.SAVED_PRESET_PATH + utils.RESEVERD_PRESET, weights_only=True)

style = {'background':'white'}
item_layout = Layout(width='auto')

protected_items = [Checkbox(layout=item_layout, description=dataset.i2c[i], indent=False, style=style) for i in range(len(dataset.i2c))]

box_layout = Layout(overflow='hidden scroll',
                    border='empty',
                    width='auto',
                    height='300px',
                    flex_flow='column',
                    display='flex',
                    padding='0')

PROTECTED_SELECTION = Box(children=protected_items, layout=box_layout)
GROUND_SELECTION = Combobox(
    value=None,
    placeholder='Choose a feature',
    options=dataset.i2c+[''],
    description='Ground truth:',
    ensure_option=True,
    style={'description_width':'150px'},
    layout=Layout(width='auto'),
)
PREDICTED_SELECTION = Combobox(
    value=None,
    placeholder='Choose a feature',
    options=dataset.i2c+[''],
    description='Predicted by model:',
    ensure_option=True,
    style={'description_width':'150px'},
    layout=Layout(width='auto'),
)
FEEDBACK_INFO_SLECETION = Label(
    value='',
)

SELECTION_BOX = VBox([
    Label('Please select the ground truth and the predicted probability (if available):'), 
    GROUND_SELECTION,
    PREDICTED_SELECTION,
    FEEDBACK_INFO_SLECETION,
    Label('Please select all protected variables:'), 
    PROTECTED_SELECTION,
    ])

## Restore data
preset_protected_values = params.get('protected_values', torch.zeros(len(dataset.i2c), dtype=bool))
for item, value in zip(protected_items, preset_protected_values):
    item.value = bool(value)
GROUND_SELECTION.value=params.get('ground_truth_column', '')
PREDICTED_SELECTION.value=params.get('prediction_column', '')

output_selection = widgets.Output()
display(SELECTION_BOX, output_selection)

def update_selection_feedback(id):
    with output_selection:
        if GROUND_SELECTION.value == PREDICTED_SELECTION.value != '':
            FEEDBACK_INFO_SLECETION.value=f'Both are now selected as "{GROUND_SELECTION.value}", Please make sure that the ground truth is not the same as the predictions'
        else:
            FEEDBACK_INFO_SLECETION.value='You have selected ' + (f'ground truth as "{GROUND_SELECTION.value}"' if GROUND_SELECTION.value else 'no ground truth') + ' and ' + (f'predictions as "{PREDICTED_SELECTION.value}"' if PREDICTED_SELECTION.value else 'no predictions')

GROUND_SELECTION.on_trait_change(update_selection_feedback)
PREDICTED_SELECTION.on_trait_change(update_selection_feedback)
SELECTION_BOX.on_widget_constructed(update_selection_feedback)

SAVE_CHANGES = Button(
    description='Save changes',
    button_style='success',
    tooltip='Save your current selection',
)
output_save_changes = widgets.Output()

def save_changes(id):
    with output_save_changes:
        params['protected_values'] = torch.tensor([protected_item.value for protected_item in protected_items])
        params['ground_truth_column'] = GROUND_SELECTION.value
        params['prediction_column'] = PREDICTED_SELECTION.value
        torch.save(params, utils.SAVED_PRESET_PATH + utils.RESEVERD_PRESET)

SAVE_CHANGES.on_click(save_changes)

display(SAVE_CHANGES, output_save_changes)

Output()

Button(button_style='success', description='Save changes', style=ButtonStyle(), tooltip='Save your current sel…

Output()

In [8]:
params = torch.load(utils.SAVED_PRESET_PATH + utils.RESEVERD_PRESET, weights_only=True)

style = {'background':'white'}
item_layout = Layout(width='auto')

fairness_groups_names = ['Predicted and actual outcomes',
                         'Predicted probibilities and actual outcomes',
                         'similarity based',
                         'Causal Reasoning']

metric_names = [
    ['Error rate', 'Predictive value', 'Statistical parity'],
    ['Test-fairness', 'Balance in classes'],
    ['Similarity']
]

VALID_GROUND = Valid(value=True, description='Ground truth')
VALID_PRED = Valid(value=True, description='Prediction')

TEST_FAIRNESS = VBox([
    Checkbox(layout=item_layout,
             value=True,
             description='Use metric?',
             indent=False),
    Label(value='Please, make a selections'),  ## label to explain
    VALID_GROUND, VALID_PRED,
])

BALANCE_IN_CLASS = VBox([
    Checkbox(layout=item_layout,
             value=True,
             description='Use metric?',
             indent=False),
    Label(value='Please, make a selections'),  ## label to explain
    Checkbox(layout=item_layout, 
                     description='Show the positive balance', 
                     indent=False, 
                     style=style),
    Checkbox(layout=item_layout, 
                     description='Show the negative balance', 
                     indent=False, 
                     style=style),
    VALID_GROUND, VALID_PRED,
])

ERROR_RATE_METRICS = VBox([
    Checkbox(layout=item_layout,
             value=True,
             description='Use metric?',
             indent=False),
    Label(value='Please, make a selections'),  ## label to explain
    VALID_GROUND, VALID_PRED,
])

PREDICTIVE_VALUE_METRICS = VBox([
    Checkbox(layout=item_layout,
             value=True,
             description='Use metric?',
             indent=False),
    Label(value='Please, make a selections'),  ## label to explain
    VALID_GROUND, VALID_PRED,
])

STATISTICAL_PARITY = VBox([
    Checkbox(layout=item_layout,
             value=True,
             description='Use metric?',
             indent=False),
    Label(value='Please, make a selections'),  ## label to explain
    VALID_PRED,
])

SIMILARITY_BASED = VBox([
    Checkbox(layout=item_layout,
             value=True,
             description='Use metric?',
             indent=False),
    Label(value='Please, make a selections'),  ## label to explain
    IntText(value=1000,
        description='Sample size:'),
    RadioButtons(options=['Manhattan Distance', 'Euclidean Distance', 'cosine'],
        value='Manhattan Distance',
        style = {'description_width': 'initial'},
        description='Choose distance metric:',
    ),
    VALID_PRED,
])

metrics_groups = [
    [ERROR_RATE_METRICS, PREDICTIVE_VALUE_METRICS, STATISTICAL_PARITY],
    [TEST_FAIRNESS, BALANCE_IN_CLASS],
    [SIMILARITY_BASED]
]

fairness_groups = [
    Accordion(children=metrics_groups[0], titles=metric_names[0]),
    Accordion(children=metrics_groups[1], titles=metric_names[1]),
    Accordion(children=metrics_groups[2], titles=metric_names[2]),
]

accordion = Accordion(children=fairness_groups, titles=fairness_groups_names)

## Restore data
default = [[metric.children[0].value for metric in group] for group in metrics_groups]
metric_selection = params.get('metric_selection', default)
for group, group_selection in zip(metrics_groups, metric_selection):
    for metric, selection in zip(group, group_selection):
        metric.children[0].value = selection
VALID_GROUND.value = True if params.get('ground_truth_column', '') else False
VALID_PRED.value = True if params.get('prediction_column', '') else False
BALANCE_IN_CLASS.children[2].value = params.get('balance_pos', True)
BALANCE_IN_CLASS.children[3].value = params.get('balance_neg', True)
SIMILARITY_BASED.children[2].value = params.get('sample_limit', 1000)
SIMILARITY_BASED.children[3].value = params.get('distance_metric', 'Manhattan Distance')

display(accordion)

SAVE_CHANGES = Button(
    description='Save changes',
    button_style='success',
    tooltip='Save your current selection',
)
output_save_changes = widgets.Output()

def save_changes(id):
    with output_save_changes:
        params['metric_selection'] = [[metric.children[0].value for metric in group] for group in metrics_groups]
        params['balance_pos'] = BALANCE_IN_CLASS.children[2].value
        params['balance_neg'] = BALANCE_IN_CLASS.children[3].value
        params['sample_limit'] = SIMILARITY_BASED.children[2].value
        params['distance_metric'] = SIMILARITY_BASED.children[3].value
        torch.save(params, utils.SAVED_PRESET_PATH + utils.RESEVERD_PRESET)

SAVE_CHANGES.on_click(save_changes)

display(SAVE_CHANGES, output_save_changes)

FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints/presets/session_save.pt'

In [6]:
# Loading and calculating each metric 
params = torch.load(utils.SAVED_PRESET_PATH + utils.RESEVERD_PRESET, weights_only=True)

def flatten(xss):
    return [x for xs in xss for x in xs]

metrics = [[Error_rate_metrics, Predictive_value_metrics, statistical_parity],
           [well_calibration, balance_in_pos_neg],
           [LipschitzFairness]]

default = [[metric.children[0].value for metric in group] for group in metrics_groups]
metric_selection = params.get('metric_selection', default)
used_metrics = [(metric(dataset, params), metric_name) for metric, metric_name, selection 
                 in zip(flatten(metrics), flatten(metric_names), flatten(metric_selection)) 
                 if selection]

In [7]:
# show results of each metric
protected_values = params.get('protected_values', torch.zeros(len(dataset.i2c), dtype=bool))
protected_names = [name for name, is_protected in zip(dataset.i2c, protected_values)
                        if is_protected]

for metric, metric_name in used_metrics:
    print(f'{'#' * (len(metric_name) + 11)}\n# Metric {metric_name} #\n{'#' * (len(metric_name) + 11)}')
    for image, name in zip(metric.show(), protected_names):
        image.show()

#####################
# Metric Error rate #
#####################


###########################
# Metric Predictive value #
###########################


#############################
# Metric Statistical parity #
#############################


########################
# Metric Test-fairness #
########################


/home/itsmick_/miniconda3/lib/python3.12/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




#############################
# Metric Balance in classes #
#############################


In [ ]:
# Saving parameters to file
OUTPUT_FILE_NAME = Text(
    placeholder='Type output file name',
    description='File name:',
    tooltip='do not forget the file extension'
)
FEEDBACK_INFO = Label(
    value='',
)
SUBMIT_BUTTON = Button(
    description='Save preset',
    disabled=True,
    button_style='danger',
    tooltip='No file name',
)

info_box = VBox([
    Label(value=f'Select a name for your preset file here, This will be saved in the folder "{utils.SAVED_PRESET_PATH}"'),
    OUTPUT_FILE_NAME,
    SUBMIT_BUTTON,
    FEEDBACK_INFO,
])

output_savefile = widgets.Output()
display(info_box, output_savefile)


def update_file_name_info(id):
    with output_savefile:
        if type(id) == Button:
            params = torch.load(utils.SAVED_PRESET_PATH + utils.RESEVERD_PRESET, weights_only=True)

            torch.save(params, utils.SAVED_PRESET_PATH + OUTPUT_FILE_NAME.value)
            FEEDBACK_INFO.value=f'Preset saved to "{OUTPUT_FILE_NAME.value}"'
        
        if type(id) == Text:
            FEEDBACK_INFO.value=''
        
        file_exits = os.path.exists(utils.SAVED_PRESET_PATH + OUTPUT_FILE_NAME.value)
        if OUTPUT_FILE_NAME.value == '':
            SUBMIT_BUTTON.button_style='danger'
            SUBMIT_BUTTON.tooltip='No file name'
            SUBMIT_BUTTON.disabled=True
        elif OUTPUT_FILE_NAME.value == 'session_save.pt':
            SUBMIT_BUTTON.button_style='danger'
            SUBMIT_BUTTON.tooltip='This file name is reserved'
            SUBMIT_BUTTON.disabled=True
        elif not OUTPUT_FILE_NAME.value.endswith('.pt'):
            SUBMIT_BUTTON.button_style='warning'
            SUBMIT_BUTTON.tooltip='Please make it a .pt file'
            SUBMIT_BUTTON.disabled=False
        elif file_exits:
            SUBMIT_BUTTON.button_style='info'
            SUBMIT_BUTTON.tooltip='A file already exists with this name, overwrite is possible'
            SUBMIT_BUTTON.disabled=False
        else:
            SUBMIT_BUTTON.button_style='success'
            SUBMIT_BUTTON.tooltip='Save file'
            SUBMIT_BUTTON.disabled=False

        
OUTPUT_FILE_NAME.on_trait_change(update_file_name_info)
SUBMIT_BUTTON.on_click(update_file_name_info)
